<a href="https://colab.research.google.com/github/Loga19818eeanvesh/Natural_Language_Processing_Sequence_Models_Assignments/blob/main/tf_c3w2_l1_IMDB_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import tensorflow as tf
print(tf.__version__)


2.7.0


In [14]:
!pip install -q tensorflow-datasets

In [15]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [16]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [17]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<UNK>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
#
vocab_size = len(word_index)
#
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type, padding='post')

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length, truncating=trunc_type, padding='post')

In [18]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [19]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 120, 16)           1417328   
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 6)                 102       
                                                                 
 dense_3 (Dense)             (None, 1)                 7         
                                                                 
Total params: 1,417,437
Trainable params: 1,417,437
Non-trainable params: 0
_________________________________________________________________


In [20]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 12s 15ms/step - loss: 0.5550 - accuracy: 0.7567 - val_loss: 0.4188 - val_accuracy: 0.8228
Epoch 2/10
782/782 [==============================] - 11s 15ms/step - loss: 0.3140 - accuracy: 0.8761 - val_loss: 0.3655 - val_accuracy: 0.8373
Epoch 3/10
782/782 [==============================] - 12s 15ms/step - loss: 0.2206 - accuracy: 0.9194 - val_loss: 0.3688 - val_accuracy: 0.8396
Epoch 4/10
782/782 [==============================] - 12s 15ms/step - loss: 0.1592 - accuracy: 0.9486 - val_loss: 0.3956 - val_accuracy: 0.8315
Epoch 5/10
782/782 [==============================] - 10s 13ms/step - loss: 0.1145 - accuracy: 0.9656 - val_loss: 0.4359 - val_accuracy: 0.8259
Epoch 6/10
782/782 [==============================] - 11s 13ms/step - loss: 0.0803 - accuracy: 0.9787 - val_loss: 0.4948 - val_accuracy: 0.8163
Epoch 7/10
782/782 [==============================] - 11s 15ms/step - loss: 0.0556 - accuracy: 0.9862 - val_loss: 0.5392 - val_accuracy:

In [23]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights) # shape: (vocab_size, embedding_dim)

[[ 0.00708602 -0.01925499  0.0424652  ... -0.01153643  0.16418557
   0.12582844]
 [ 0.03560961 -0.03044878  0.01728338 ...  0.0293369   0.00375647
   0.0023177 ]
 [-0.07078787 -0.11636305  0.10380837 ...  0.10576101  0.35253614
   0.2424144 ]
 ...
 [-0.1549196  -0.099542   -0.0533168  ...  0.1305487  -0.09591273
  -0.05766515]
 [-0.09107854 -0.11800742 -0.11230761 ...  0.1039628  -0.08006514
  -0.08279612]
 [-0.03188276 -0.00835538 -0.04490591 ... -0.03878934  0.04223228
   0.00735857]]


In [24]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [25]:
try:
    from google.colab import files
except ImportError:
    pass
else:
    files.download('vecs.tsv')
    files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
